In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
samples=["1305912","1313281","1219699","1305906","1312819","1217423","1313275","1135667","1228994","1135670","1232081","1308756","1219476","1229000","1232087","1308762","1130761","1130764","1217426","1219693"]

for i in samples:
    adata = sc.read_10x_h5('../outs/'+i+'/cb_'+i+'_0_fpr_filtered.h5')
# for each cell compute fraction of counts in mito genes vs. all genes
    # add the total counts per cell as observations-annotation to adata
    
    adata.var["mt"] = adata.var_names.str.startswith("MT-")
    # ribosomal genes
    adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
    # hemoglobin genes.
    adata.var["hb"] = adata.var_names.str.contains(("^HB[^(P)]"))
    sc.pp.calculate_qc_metrics(
        adata, qc_vars=["mt", "ribo", "hb"], inplace=True, percent_top=[20], log1p=True)
    adata.obs["outlier"] = adata.obs["n_genes_by_counts"] < 200
    adata.obs["mt_outlier"] = adata.obs["pct_counts_mt"] > 5
        
    print(adata)
    
    adata = adata[(~adata.obs.outlier) & (~adata.obs.mt_outlier)].copy()
    
    print(adata)
    print('making gene expr file')
    adata.to_df().to_csv(i+"_gene_expression.csv")
    print('making metadata file')
    adata.obs.to_csv(i+"_metadata.csv")